# Importing shapefiles into GOSTnets

This script is experimenting with converting shapefiles to G objects. The solution works because the input shapefile is derived from an existing OSM file and comes with from and to nodes.

In [100]:
import os, sys, importlib

import pandas as pd
import geopandas as gpd
import networkx as nx

from shapely.wkt import loads
from shapely.geometry import Point
from random import sample 

## Your GOSTnets path could be different
#sys.path.append(r"C:\Work\Code\GOSTnets")
sys.path.append("../../../GOSTNets")
import GOSTnets as gn

In [101]:
# Read in input shapefile and convert to G object
input_shapefile = "cxb_roads_edit.shp"

inD = gpd.read_file(input_shapefile)
G = nx.from_pandas_edgelist(inD, edge_attr=['length', 'infra_type', "osm_id", 'geometry'], source="stnode", target="endnode")
G = G.to_directed()

In [102]:
# the nodes in the dataset do not have coordinates, so let's fix that
edges = list(G.edges(data=True))
nodes = G.nodes(data=True)
all_nodes = []
# Loop through all the nodes to extract their coordinates from the nodes
for n in nodes:
    # For the current node, loop through the edges until we find an edge witht he current node
    found_node = False
    edge_count = 0
    while not found_node:
        e = edges[edge_count]
        edge_count = edge_count + 1        
        # if the current node is part of the current edge, we can extract the coordinate        
        if n[0] in e: 
            found_node = True
            # The coordinate for the node is either the first or final coordinate of the current edge            
            pt_idx = 0
            if e.index(n[0]) == 1:
                pt_idx = -1
            #Extract the appropriate point and store the new node
            pt = list(e[2]['geometry'].coords)[pt_idx]
            node_vals = {'x':pt[0], 'y':pt[1]}
            all_nodes.append([n[0], node_vals])
            #G.remove_node(n[0])
            #G.add_node(n[0], **node_vals)
G.update(nodes=all_nodes)

In [103]:
# inspect the resulting Graph
nodes = list(G.nodes(data=True))
edges = list(G.edges(data=True))
print(len(nodes))
print(nodes[0])
print(len(edges))
print(edges[0])

7922
(0, {'x': 92.3205152, 'y': 20.827246})
18047
(0, 4690, {'length': 0.128829058995489, 'infra_type': 'unclassified', 'osm_id': '644427970', 'geometry': <shapely.geometry.linestring.LineString object at 0x1313e6950>})


In [104]:
type(G)

networkx.classes.digraph.DiGraph

In [105]:
#list_of_graphs[1].number_of_edges()

In [106]:
"""
print('print G before')
print(G.size())
# save the largest subgraph for networkX 2.4
largest = max(nx.strongly_connected_components(G), key=len)
G = nx.induced_subgraph(G,largest).copy()
print('print G after')
print(G.size())
"""

"\nprint('print G before')\nprint(G.size())\n# save the largest subgraph for networkX 2.4\nlargest = max(nx.strongly_connected_components(G), key=len)\nG = nx.induced_subgraph(G,largest).copy()\nprint('print G after')\nprint(G.size())\n"

In [107]:
type(G)

networkx.classes.digraph.DiGraph

In [108]:
#gn.save(G,'G_largest_graph','./',pickle=False,nodes=False,edges=True)

In [109]:
#nx.strongly_connected_components(G)

In [110]:
# Identify only the largest graph

#list_of_subgraphs = list(nx.strongly_connected_component_subgraphs(G))
list_of_subgraphs = list(G.subgraph(c).copy() for c in nx.strongly_connected_components(G))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i
G = max_graph


In [111]:
gn.save(G,'G_largest_graph2','./',pickle=False,nodes=False,edges=True)

In [112]:
# inspect the resulting Graph
nodes = list(G.nodes(data=True))
edges = list(G.edges(data=True))
print(len(nodes))
print(nodes[0])
print(len(edges))
print(edges[0])

4993
(0, {'x': 92.3205152, 'y': 20.827246})
11843
(0, 4690, {'length': 0.128829058995489, 'infra_type': 'unclassified', 'osm_id': '644427970', 'geometry': <shapely.geometry.linestring.LineString object at 0x1313e6950>})


In [77]:
# read in origins and destinations
#input_folder = r"C:\Work\Code\GOST_PublicGoods\Implementations\RobertBanick_ShapefileTToG\SampleData"
input_folder = r"./input_data"
origins_file = os.path.join(input_folder, 'hrsl_pts_admins.csv')
destinations_file  = os.path.join(input_folder, 'education_cxb/education_cxb_lged.shp')

inO = pd.read_csv(origins_file)
inD = gpd.read_file(destinations_file)
inD = inD.to_crs({'init': 'epsg:4326'})

/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [78]:
len(inO)

120498

In [79]:
#Convert the input origins to a POINT geodaframe
inO_geom = [loads(x) for x in inO['geometry']]
inO_geom = [Point(x[0].x, x[0].y) for x in inO_geom]

inO = gpd.GeoDataFrame(inO.drop(['geometry'], axis=1), geometry = inO_geom, crs = {'init': 'epsg:4326'})

# sample origins for testing
# inO = inO.loc[sample(list(inO.index), 1000)]

In [80]:
print(inO.shape)
print(inO.crs)
print(inD.shape)
print(inD.crs)

(120498, 8)
{'init': 'epsg:4326'}
(788, 20)
{'init': 'epsg:4326'}


In [81]:
origins = gn.pandana_snap(G, inO)

/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [82]:
destinations = gn.pandana_snap(G, inD)

/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/thomasgertin3/anaconda3/envs/gostnets2/lib/python3.7/site-packages/pyproj-2.4.2.post1-py3.7-macosx-10.9-x86_64.egg/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [94]:
origins.loc[origins['NN'] == 3140]

,hrsl_ID,VALUE,Upaz_name,Upz_UID,index_right,ADM4_EN,ADM4_PCODE,geometry,NN,NN_dist
42187,67318,26.443955,Maheshkhali Upazila,202249,2446,Kalarmarchhara,20224959,POINT (91.91722 21.66667),3140,0.156716
42191,67366,26.443955,Maheshkhali Upazila,202249,2446,Kalarmarchhara,20224959,POINT (91.91444 21.66639),3140,0.158914
42192,67367,26.443955,Maheshkhali Upazila,202249,2446,Kalarmarchhara,20224959,POINT (91.91833 21.66639),3140,0.155632
42193,67368,26.443955,Maheshkhali Upazila,202249,2446,Kalarmarchhara,20224959,POINT (91.91972 21.66639),3140,0.154466
42197,67395,26.443955,Maheshkhali Upazila,202249,2446,Kalarmarchhara,20224959,POINT (91.91444 21.66611),3140,0.158767
...,...,...,...,...,...,...,...,...,...,...
82583,76657,35.005145,Cox's Bazar Sadar Upazila,202224,2162,Islamabad,20222442,POINT (92.04694 21.57917),3140,0.003658
82600,76688,35.005145,Cox's Bazar Sadar Upazila,202224,2162,Islamabad,20222442,POINT (92.04667 21.57889),3140,0.004048
82601,76689,35.005145,Cox's Bazar Sadar Upazila,202224,2162,Islamabad,20222442,POINT (92.04694 21.57889),3140,0.003883
82602,76690,35.005145,Cox's Bazar Sadar Upazila,202224,2162,Islamabad,20222442,POINT (92.04861 21.57889),3140,0.003240


In [91]:
destinations

,GPS_ID,MARK,X_LOC,Y_LOC,DATE,SCH_TYPE,SCH_NAME,SCH_B_ADDR,VILL_NAME,POST,...,ID,DIST_NAME,UPZ_CODE,UPZ_NAME,SCH_CODE,NEW_ID,PathName,geometry,NN,NN_dist
0,40351009,A51009,92.18917,21.78888,2006-09-11,A,T.T. AND D.C.,"?V,?V,H? ?X?p",T.T. AND D.C.,LAMA,...,12911.0,BANDARBAN,41505,LAMA,415050105,080220P,Chittagong\Bandarban\Lama,POINT (92.19228 21.78558),2921,0.181058
1,40351010,B51010,92.18083,21.81805,2006-09-11,B,None,hsg­l­wN­ M¦<­ee­ rl?Sx,None,None,...,12912.0,BANDARBAN,41505,LAMA,415050105,080220P,Chittagong\Bandarban\Lama,POINT (92.18466 21.80892),2921,0.194793
2,40351011,A51011,92.14388,21.87195,2006-09-11,A,CHAMPA TALI,Q­?f­am?,CHAMPA TALI,LAMA,...,12913.0,BANDARBAN,41505,LAMA,415050106,080220P,Chittagong\Bandarban\Lama,POINT (92.19078 21.77416),2921,0.171579
3,40351015,A51015,92.18778,21.82417,2006-09-11,A,KRONG THONG PARA,oýwbw f­s­,KRONG THONG PARA,LAMA,...,12917.0,BANDARBAN,41505,LAMA,415050108,080220P,Chittagong\Bandarban\Lama,POINT (92.19403 21.81968),2921,0.208987
4,40351056,A51056,92.06417,21.90305,2006-09-11,A,CHAMBI,Q­?d,CHAMBI,AZIZ NAGAR,...,12958.0,BANDARBAN,41505,LAMA,415050401,080220P,Chittagong\Bandarban\Lama,POINT (92.07054 21.90111),2921,0.252039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,32614023,A94023,92.13048,21.28923,2006-08-21,A,TALI PARA,-a­?m f­s­,TALI PARA,RATNA PALONG,...,10622.0,COX'S BAZAR,41201,UKHIYA,412010204,None,Chittagong\Cox's Bazar\UKHIYA,POINT (92.13120 21.28571),1015,0.003596
784,32614021,C94021,92.13768,21.28437,2006-08-21,C,BALUKHIA,h­m?Mu­,BALUKHIA,CHAK BAITA,...,10620.0,COX'S BAZAR,41201,UKHIYA,412010203,None,Chittagong\Cox's Bazar\UKHIYA,POINT (92.13842 21.28085),3190,0.000743
785,32614020,B94020,92.09893,21.27250,2006-08-21,B,None,f?OQj laA­ rl?Sx fD­x ?hx,None,None,...,10619.0,COX'S BAZAR,41201,UKHIYA,412010202,None,Chittagong\Cox's Bazar\UKHIYA,POINT (92.09961 21.26893),9036,0.001192
786,32614068,A94068,92.12432,21.26840,2006-08-21,A,UTTAR PUKURIA,Esl fL?lu­,UTTAR PUKURIA,UKHIYA,...,10667.0,COX'S BAZAR,41201,UKHIYA,412010409,None,Chittagong\Cox's Bazar\UKHIYA,POINT (92.12505 21.26484),8321,0.000618


In [84]:
oNodes = origins['NN'].unique()
dNodes = destinations['NN'].unique()

In [85]:
od = gn.calculate_OD(G, oNodes, dNodes, -1)
od.shape

(3005, 325)

In [86]:
od

array([[  0.,  85.,   5., ...,  91.,  84.,  91.],
       [ 25.,  90.,  22., ...,  89.,  73.,  81.],
       [  4.,  83.,   3., ...,  89.,  82.,  89.],
       ...,
       [181., 135., 178., ..., 126., 110., 118.],
       [179., 133., 176., ..., 124., 108., 116.],
       [178., 132., 175., ..., 123., 107., 115.]])

In [87]:
OD_df = pd.DataFrame(od, columns = dNodes, index = oNodes)

In [88]:
OD_df[:500]

,2921,7404,5451,7380,2428,5377,1391,229,2698,5505,...,5371,1789,6058,5249,8667,2100,1015,3190,9036,8321
2921,0.0,85.0,5.0,1.0,2.0,3.0,43.0,45.0,44.0,41.0,...,89.0,90.0,91.0,93.0,91.0,86.0,90.0,91.0,84.0,91.0
3210,25.0,90.0,22.0,24.0,23.0,22.0,24.0,26.0,25.0,22.0,...,79.0,85.0,96.0,89.0,96.0,91.0,90.0,89.0,73.0,81.0
3140,4.0,83.0,3.0,3.0,2.0,1.0,41.0,43.0,42.0,39.0,...,87.0,88.0,89.0,91.0,89.0,84.0,88.0,89.0,82.0,89.0
2428,2.0,83.0,3.0,1.0,0.0,1.0,41.0,43.0,42.0,39.0,...,87.0,88.0,89.0,91.0,89.0,84.0,88.0,89.0,82.0,89.0
3642,21.0,94.0,18.0,20.0,19.0,18.0,52.0,54.0,53.0,50.0,...,98.0,99.0,100.0,102.0,100.0,95.0,99.0,100.0,93.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6310,96.0,20.0,93.0,95.0,94.0,93.0,63.0,72.0,70.0,65.0,...,11.0,16.0,20.0,13.0,24.0,22.0,22.0,18.0,13.0,13.0
9254,97.0,17.0,94.0,96.0,95.0,94.0,64.0,73.0,71.0,66.0,...,12.0,14.0,17.0,10.0,21.0,19.0,19.0,15.0,14.0,12.0
340,96.0,16.0,93.0,95.0,94.0,93.0,63.0,72.0,70.0,65.0,...,11.0,13.0,16.0,9.0,20.0,18.0,18.0,14.0,13.0,11.0
2555,93.0,10.0,90.0,92.0,91.0,90.0,72.0,81.0,79.0,74.0,...,16.0,16.0,14.0,12.0,18.0,20.0,21.0,17.0,22.0,14.0


In [89]:
OD_df.to_csv("./output_csv2.csv")

In [90]:
'''# Write the largest graph back to file
all_data = {}
idx=0
for o, d, data in G.edges(data=True):
    data['o'] = o
    data['d'] = d
    all_data[idx] = data    
    idx = idx+1
big_graph = gpd.GeoDataFrame(all_data).transpose()
big_graph.to_csv(input_shapefile.replace(".shp", "_big_boy.csv"))
big_graph.head()'''

'# Write the largest graph back to file\nall_data = {}\nidx=0\nfor o, d, data in G.edges(data=True):\n    data[\'o\'] = o\n    data[\'d\'] = d\n    all_data[idx] = data    \n    idx = idx+1\nbig_graph = gpd.GeoDataFrame(all_data).transpose()\nbig_graph.to_csv(input_shapefile.replace(".shp", "_big_boy.csv"))\nbig_graph.head()'